## Import

In [1]:
%load_ext autoreload
%autoreload 2



In [2]:
from pathlib import Path
import time

In [ ]:
from pysmFISH.pipeline import Pipeline

Matplotlib is building the font cache; this may take a moment.
/home/simone/mini/envs/test_d_seg/lib/python3.8/site-packages/dask_jobqueue/core.py:19: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/home/simone/mini/envs/test_d_seg/lib/python3.8/site-packages/dask_jobqueue/core.py:19: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/home/simone/mini/envs/test_d_seg/lib/python3.8/site-packages/dask_jobqueue/htcondor.py:6: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import parse_bytes


## Prepare the folders for experimental tests

In [ ]:
!mkdir 'processing_folder/LBEXP20210718_EEL_Mouse_448_2'
!cp -r 'data_storage/LBEXP20210718_EEL_Mouse_448_2/fresh_tissue' 'processing_folder/LBEXP20210718_EEL_Mouse_448_2/'
!cp -r 'data_storage/required_folders_storage/codebooks' 'processing_folder/'
!cp -r 'data_storage/required_folders_storage/probes_sets' 'processing_folder/'
!cp -r 'data_storage/LBEXP20210718_EEL_Mouse_448_2/210906_21_07_09_LBEXP20210718_EEL_Mouse_448_2_img_data_dataset.parquet' 'processing_folder/LBEXP20210718_EEL_Mouse_448_2/'

## Initialize the pipeline

In [ ]:
# Enter the required parameters
experiment_fpath = Path('/Users/simone.codeluppi/Documents/data_analysis_jlabs_sc/notebooks_pysmFISH/processing_folder/LBEXP20210718_EEL_Mouse_448_2')
dataset_name = '210906_21_07_09_LBEXP20210718_EEL_Mouse_448_2_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'local'

In [ ]:
%%time
# Because you are running the pipeline locally you should define the number of cores and memory/core
# 3-4 GB / code should be safe for running the processing
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        cores=3,
        memory='12GB',
        chunk_size = 6,
        dataset_path = experiment_fpath / dataset_name)

## Run the required steps

In [ ]:
%%time
running_pipeline.run_setup()
running_pipeline.run_cluster_activation()
running_pipeline.run_parsing()
running_pipeline.run_required_steps()

In [ ]:
running_pipeline.client

## Process the images of the fresh tissue

In [ ]:
%%time
running_pipeline.processing_fresh_tissue_step(parsing=False)

In [ ]:
running_pipeline.client.close()

In [ ]:
running_pipeline.cluster.close()

## Remove overlapping objects

In [ ]:
from pysmFISH.fov_processing import segmentation_NN_fov
from pysmFISH.io import load_general_zarr
import zarr
from pysmFISH.utils import convert_from_uint16_to_float64
import pickle
from pathlib import Path
import pandas as pd
from pysmFISH.stitching import organize_square_tiles
from skimage import measure
import numpy as np
from pysmFISH.data_models import Dataset
from dask.distributed import Client, LocalCluster
from pysmFISH.processing_cluster_setup import local_cluster_setup
from pysmFISH.segmentation import remove_overlapping_obj

In [ ]:
cores = 4
memory = 6
cluster = local_cluster_setup(cores=cores,memory=memory)
client = Client(cluster)

In [ ]:
client = Client()

In [ ]:
client

In [ ]:
- Bring in the information from the experiment.dataset to fill the blanks and the nuclei
- Add the creation of the segmentation folder to the path
- adjust to the segmentation_output from the experiment-path in the registered coords obj
- maybe add in the pipeline attributes the size of the overlapping objects to remove
- check with Lars for segmentation data

In [ ]:
def load_segmented_data(fov_subdataset,experiment_path):
    experiment_path = Path(experiment_path)
    experiment_name = fov_subdataset.experiment_name
    segmented_file_path = experiment_path / 'fresh_tissue' / 'fresh_tissue_segmented_nuclei_data.zarr'
    st = zarr.DirectoryStore(segmented_file_path)
    root = zarr.group(store=st,overwrite=False)
    grp_name = (
            experiment_name
            + "_segmetented_fresh_tissue_fov_"
            + str(fov_subdataset.fov_num)
        )
        
    dataset_name = "segmentation_mask_fov_" + str(fov_subdataset.fov_num)
    
    mask = root[grp_name][dataset_name][...]
    return mask

In [ ]:
def load_stitched_segmented_data(fov_subdataset,segmentation_output_path):
    segmented_region_img = pickle.load(open(segmentation_output_path / ('registered_objs_dict_fov_' + str(fov_subdataset.fov_num) + '.pkl'), 'rb'))

    obj_segmented_img = {el:coords_dict['stitched_coords'] for (el,coords_dict) in segmented_region_img.items()}
    return obj_segmented_img

def load_stitched_segmented_data_full_dict(fov_subdataset,segmentation_output_path):
    segmented_region_dict = pickle.load(open(segmentation_output_path / ('registered_objs_dict_fov_' + str(fov_subdataset.fov_num) + '.pkl'), 'rb'))
    return segmented_region_dict
    

In [ ]:
def stitch_using_coords_general_segmented_objects(fov,obj_dict,tile_corners_coords_pxl,reference_corner_fov_position,metadata):
    """
    Tiles are placed directly on the position indicated by the microscope
    coords
    """

    r_coords = tile_corners_coords_pxl[fov,0]
    c_coords = tile_corners_coords_pxl[fov,1]
    
    if obj_dict:
        
        if reference_corner_fov_position == 'top-left':
            for el,coords_dict in obj_dict.items():
                coords_dict['stitched_coords'] = np.vstack([r_coords + coords_dict['original_coords'][:,0],
                                                            c_coords + coords_dict['original_coords'][:,1]]).T
                coords_dict['stitched_centroid'] = np.vstack([r_coords + coords_dict['original_centroid'][0],
                                                            c_coords + coords_dict['original_centroid'][1]]).T

        elif reference_corner_fov_position == 'top-right':
            for el,coords_dict in obj_dict.items():
                coords_dict['stitched_coords'] = np.vstack([r_coords + coords_dict['original_coords'][:,0],
                                                            c_coords - (metadata['img_width'] -coords_dict['original_coords'][:,1])]).T
                coords_dict['stitched_centroid'] = np.vstack([r_coords + coords_dict['original_centroid'][0],
                                                            c_coords - (metadata['img_width'] -coords_dict['original_centroid'][1])]).T

        elif reference_corner_fov_position == 'bottom_left':
            for el,coords_dict in obj_dict.items():
                coords_dict['stitched_coords'] = np.vstack([r_coords + (metadata['img_height'] -coords_dict['original_coords'][:,0]),
                                                            c_coords + coords_dict['original_coords'][:,1]]).T
                coords_dict['stitched_centroid'] = np.vstack([r_coords + (metadata['img_height'] -coords_dict['original_centroid'][0]),
                                                            c_coords + coords_dict['original_centroid'][1]]).T

    return obj_dict

In [ ]:
def register_coords_obj(fov_subdataset, experiment_fpath, segmentation_output_path,
                        tile_corners_coords_global_pxl,metadata,reference_corner_fov_position='top-right'):
    
    mask = load_segmented_data(fov_subdataset,experiment_fpath)
    segmentation_output_path = Path(experiment_fpath) / 'fresh_tissue' / 'segmentation'
    
    segmented_regions = measure.regionprops(mask)
    segmented_regions_dict = {}
    for prop in segmented_regions:
        segmented_regions_dict[str(fov_subdataset.fov_num)+'-'+str(prop.label)] = {}
        segmented_regions_dict[str(fov_subdataset.fov_num)+'-'+str(prop.label)]['original_coords']=prop.coords
        segmented_regions_dict[str(fov_subdataset.fov_num)+'-'+str(prop.label)]['stitched_coords']= np.nan
        segmented_regions_dict[str(fov_subdataset.fov_num)+'-'+str(prop.label)]['original_centroid']= prop.centroid
        segmented_regions_dict[str(fov_subdataset.fov_num)+'-'+str(prop.label)]['stitched_centroid']= prop.centroid
        segmented_regions_dict[str(fov_subdataset.fov_num)+'-'+str(prop.label)]['area']= prop.area
        segmented_regions_dict = stitch_using_coords_general_segmented_objects(fov_subdataset.fov_num,segmented_regions_dict,
                                                                         tile_corners_coords_global_pxl,reference_corner_fov_position,metadata)
    pickle.dump(segmented_regions_dict,open(segmentation_output_path / ('registered_objs_dict_fov_' + str(fov_subdataset.fov_num) + '.pkl'), 'wb'))
#    return segmented_regions_dict

In [ ]:
dataset_name = '220415_16_38_11_20210718_124116_843__ChannelCy3_Nuclei_Seq0001_img_data_dataset.parquet'
dataset_nuclei = Dataset()
dataset_nuclei.load_dataset(Path(experiment_fpath) / 'fresh_tissue' / dataset_name)
dataset_nuclei.dataset['processing_type'] = 'undefined'
dataset_nuclei.dataset['overlapping_percentage'] = 5
dataset_nuclei.dataset['machine'] = 'ROBOFISH2'

metadata_nuclei = dataset_nuclei.collect_metadata(dataset_nuclei.dataset)
fov_subdataset = dataset_nuclei.dataset.iloc[0]
tile_corners_coords_global_pxl = pickle.load(open(Path(experiment_fpath) / 'fresh_tissue' / 'results' / 'global_stitched_coords.pkl','rb'))
nuclei_org_tiles = organize_square_tiles(experiment_fpath,dataset_nuclei.dataset,metadata_nuclei,round_num=1)
nuclei_org_tiles.run_tiles_organization()
a = nuclei_org_tiles.tile_corners_coords_pxl
nuclei_org_tiles.tile_corners_coords_pxl = tile_corners_coords_global_pxl
nuclei_org_tiles.determine_overlapping_regions()
reference_corner_fov_position = nuclei_org_tiles.reference_corner_fov_position

In [ ]:
mc = np.load('/Users/simone.codeluppi/Downloads/microscope_tile_corners_coords_pxl.npy')
gb = pickle.load(open('/Users/simone.codeluppi/Downloads/global_stitched_coords.pkl','rb'))
gs = pickle.load(open('/Users/simone.codeluppi/Downloads/stitching_global_shift.pkl','rb'))

In [ ]:
mc.shape

In [ ]:
mc[0,:]

In [ ]:
gb[0,:]

In [ ]:
gs[0:20,:]

In [ ]:
nuclei_org_tiles.tile_corners_coords_pxl[0,:]

In [ ]:
tile_corners_coords_global_pxl[0,:]

In [ ]:
all_futures = []
    for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
        future = client.submit(register_coords_obj,fov_subdataset,
                               experiment_fpath,
                               segmentation_output_path,
                              tile_corners_coords_global_pxl,
                              metadata_nuclei,
                               reference_corner_fov_position='top-right')
        all_futures.append(future)
    _ = client.gather(all_futures)
    

In [ ]:
from pysmFISH.segmentation import register_coords_obj

In [ ]:

    all_futures = []
    for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
        future = client.submit(register_coords_obj,fov_subdataset,
                               experiment_fpath,
                               segmentation_output_path,
                              tile_corners_coords_global_pxl,
                              metadata_nuclei,
                               reference_corner_fov_position='top-right')
        all_futures.append(future)
    _ = client.gather(all_futures)
    
    

In [ ]:
def create_label_image(experiment_fpath,
                       segmentation_output_path,
                       ds_nuclei,
                       nuclei_org_tiles,
                       nuclei_adjusted_coords,
                       client,
                       min_overlapping_pixels_segmentation = 20):
    
    # Get objects properties and register to the data
    nuclei_org_tiles.tile_corners_coords_pxl = tile_corners_coords_global_pxl
    reference_corner_fov_position = nuclei_org_tiles.reference_corner_fov_position
        
    
    
    # Remove overlapping objects paralle
    all_futures = []
    for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
        future = client.submit(remove_overlapping_obj,
                               fov_subdataset,
                               nuclei_org_tiles,
                              segmentation_output_path,
                              min_overlapping_pixels_segmentation)
        all_futures.append(future)
    all_outputs = client.gather(all_futures)
    
    
    # tmp_saving
    pickle.dump(all_outputs,open(segmentation_output_path / ('output_removal_tmp.pkl'), 'wb'))
    
    all_obj_original = {}
    for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
        segmented_region_img = load_stitched_segmented_data_full_dict(fov_subdataset,segmentation_output_path)
        all_obj_original.update(segmented_region_img)

    all_remove_objs_list = []
    all_add_objs = {}
    for rem, obj in all_outputs:
        all_remove_objs_list.append(rem)
        all_add_objs.update(obj)
    all_remove_objs_list = [val for el in all_remove_objs_list for val in el ]

    for cell_id, coords in all_add_objs.items():
        all_obj_original[cell_id]['stitched_coords'] = coords

    all_remove_objs_list_tmp=all_remove_objs_list.copy()
    for obj in all_obj_original:
        if obj in all_remove_objs_list_tmp:
            all_remove_objs_list_tmp.remove(obj)

    for obj in all_remove_objs_list_tmp:
        all_obj_original.pop(obj,None)
        
    
    pickle.dump(all_obj_original,open(segmentation_output_path / ('segmented_objects_dict.pkl'), 'wb'))
    
    # Create image
    # Calculate image size 

    row_min = tile_corners_coords_global_pxl[:,0].min()
    col_min = tile_corners_coords_global_pxl[:,1].min()
    row_max = tile_corners_coords_global_pxl[:,0].max()
    col_max = tile_corners_coords_global_pxl[:,1].max()

    nrows = np.int(row_max - row_min + metadata_nuclei['img_height'])
    ncols = np.int(col_max - col_min + metadata_nuclei['img_width'])

    img = np.zeros([nrows,ncols])

    for idx, data_dict in enumerate(all_obj_original.items()):
        coords = data_dict[1]['stitched_coords']
        coords = coords -[row_min,col_min]
        coords = coords.astype(int)
        img[coords[:,0],coords[:,1]] = idx

    zarr_fpath = segmentation_output_path / 'image_segmented_labels.zarr'
    store = zarr.DirectoryStore(zarr_fpath,'w')
    grp = zarr.group(store=store,overwrite=True)
    grp.create_dataset(name='segmented_labels_image', data=img)

In [ ]:
experiment_fpath = Path('/Users/simone.codeluppi/Documents/data_analysis_jlabs_sc/notebooks_pysmFISH/processing_folder/LBEXP20210718_EEL_Mouse_448_2')
segmentation_output_path = experiment_fpath / 'fresh_tissue' / 'segmentation'

In [ ]:
from pysmFISH.segmentation import load_stitched_segmented_data_full_dict

In [ ]:
%time
create_label_image(experiment_fpath,
                       segmentation_output_path,
                       dataset_nuclei,
                       nuclei_org_tiles,
                       tile_corners_coords_global_pxl,
                       client)

In [ ]:
def create_low_mag_beads_target(experiment_fpath):
    all_counts_collected = []
    counts_paths = (Path(experiment_fpath) / 'fresh_tissue' /'results').glob('*counts_beads_fresh_tissue_decoded_fov*')
    for count_fpath in counts_paths:
        data = pd.read_parquet(count_fpath)
        data = data.loc[:, ['r_px_global_stitched_nuclei','c_px_global_stitched_nuclei']]
        all_counts_collected.append(data)
    all_counts_collected = pd.concat(all_counts_collected,axis=0)
    target = all_counts_collected.loc[:,['r_px_global_stitched_nuclei','c_px_global_stitched_nuclei']].to_numpy()
    return target

In [ ]:
beads_df = create_low_mag_beads_target(experiment_fpath)


In [ ]:
np.save(segmentation_output_path /'beads-40X.npy',beads_df)

In [ ]:
def create_high_mag_beads_RNA_source(experiment_path,hamming_distance=3):
    all_beads = []
    all_RNA = []
    filter_distance = hamming_distance / 16
    counts_paths = (Path(experiment_path) / 'results').glob('*_decoded_fov_*')
    for count_fpath in counts_paths:
        data = pd.read_parquet(count_fpath)
        data_beads = data.loc[(data.channel == 'Europium') &
                        (data.mapped_beads_type == 'large') &
                        (data.round_num == 1), ['r_px_global_stitched','c_px_global_stitched']]
        
        data_RNA = data.loc[(data.channel != 'Europium') &
                            (data.hamming_distance < filter_distance), ['decoded_genes','r_px_global_stitched','c_px_global_stitched']]
        all_beads.append(data_beads)
        all_RNA.append(data_RNA)
    all_beads = pd.concat(all_beads,axis=0)
    all_RNA = pd.concat(all_RNA,axis=0)
    source_beads = all_beads.loc[:,['r_px_global_stitched','c_px_global_stitched']].to_numpy()
    source_RNA_df = all_RNA.loc[:,['decoded_genes','r_px_global_stitched','c_px_global_stitched']].to_numpy()
    return source_beads, source_RNA_df

In [ ]:
source_beads, source_RNA_df = create_high_mag_beads_RNA_source(experiment_fpath,hamming_distance=3)

In [ ]:
np.save(Path(experiment_fpath) / 'results' /'beads-60X-global.npy',beads_df)

In [ ]:
def create_high_mag_beads_RNA_source_micro(experiment_path,hamming_distance=3):
    all_beads = []
    all_RNA = []
    filter_distance = hamming_distance / 16
    counts_paths = (Path(experiment_path) / 'results').glob('*_decoded_fov_*')
    for count_fpath in counts_paths:
        data = pd.read_parquet(count_fpath)
        data_beads = data.loc[(data.channel == 'Europium') &
                        (data.mapped_beads_type == 'large') &
                        (data.round_num == 1), ['r_px_microscope_stitched','c_px_microscope_stitched']]
        
        data_RNA = data.loc[(data.channel != 'Europium') &
                            (data.hamming_distance < filter_distance), ['decoded_genes','r_px_microscope_stitched','c_px_microscope_stitched']]
        all_beads.append(data_beads)
        all_RNA.append(data_RNA)
    all_beads = pd.concat(all_beads,axis=0)
    all_RNA = pd.concat(all_RNA,axis=0)
    source_beads = all_beads.loc[:,['r_px_microscope_stitched','c_px_microscope_stitched']].to_numpy()
    source_RNA_df = all_RNA.loc[:,['decoded_genes','r_px_microscope_stitched','c_px_microscope_stitched']].to_numpy()
    return source_beads, source_RNA_df

In [ ]:
source_beads, source_RNA_df = create_high_mag_beads_RNA_source_micro(experiment_fpath,hamming_distance=3)

In [ ]:
source_beads.shape

In [ ]:
np.save(Path(experiment_fpath) / 'results' /'beads-60X-microscope.npy',beads_df)

In [ ]:
df = pd.read_parquet(Path(experiment_fpath) / 'results' / 'LBEXP20210718_EEL_Mouse_448_2_decoded_fov_554.parquet')

In [ ]:
df.loc[(df.channel == 'Europium')&
       (df.round_num == 1) &
       (df.mapped_beads_type,:]

In [ ]:
df.mapped_beads_type.unique()

In [ ]:
df.columns

In [ ]:
all_counts_collected = []
counts_paths = (Path('/Users/simone.codeluppi/Downloads/') /'results').glob('*counts_beads_fresh_tissue_decoded_fov*')
for count_fpath in counts_paths:
    data = pd.read_parquet(count_fpath)
    data = data.loc[:, ['r_px_original','c_px_original']]
    all_counts_collected.append(data)
all_counts_collected = pd.concat(all_counts_collected,axis=0)
target = all_counts_collected.loc[:,['r_px_original','c_px_original']].to_numpy()

In [ ]:
target.shape

In [ ]:
seg = pickle.load(open('/Users/simone.codeluppi/Downloads/segmented_objects_dict.pkl','rb' ))

In [ ]:
# Serial
for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
    register_coords_obj(fov_subdataset,experiment_fpath,tile_corners_coords_global_pxl,metadata_nuclei,reference_corner_fov_position='top-right')

In [ ]:
# Parallel
all_futures = []
for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
    future = client.submit(register_coords_obj,fov_subdataset,
                           experiment_fpath,
                           segmentation_output_path,
                          tile_corners_coords_global_pxl,
                          metadata_nuclei,
                           reference_corner_fov_position='top-right')
    all_futures.append(future)
_ = client.gather(all_futures)

In [ ]:
# Remove overlapping objects serial
segmentation_output_path = Path(experiment_fpath) / 'fresh_tissue' / 'segmentation'
min_overlapping_pixels_segmentation = 20
all_output = []
for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
    all_output.append(remove_overlapping_obj(
                           fov_subdataset,
                           nuclei_org_tiles,
                          segmentation_output_path,
                          min_overlapping_pixels_segmentation))


In [ ]:
# Remove overlapping objects paralle
min_overlapping_pixels_segmentation = 20
all_futures = []
for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
    future = client.submit(remove_overlapping_obj,
                           fov_subdataset,
                           nuclei_org_tiles,
                          segmentation_output_path,
                          min_overlapping_pixels_segmentation)
    all_futures.append(future)
all_outputs = client.gather(all_futures)


In [ ]:
# tmp_saving
pickle.dump(all_output,open(segmentation_output_path / ('output_removal_tmp.pkl'), 'wb'))

In [ ]:
all_obj_original = {}
for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
    segmented_region_img = load_stitched_segmented_data_full_dict(fov_subdataset,segmentation_output_path)
    all_obj_original.update(segmented_region_img)

In [ ]:
all_obj_original = {}
for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
    segmented_region_img = load_stitched_segmented_data_full_dict(fov_subdataset,segmentation_output_path)
    all_obj_original.update(segmented_region_img)

all_remove_objs_list = []
all_add_objs = {}
for rem, obj in all_output:
    all_remove_objs_list.append(rem)
    all_add_objs.update(obj)
all_remove_objs_list = [val for el in all_remove_objs_list for val in el ]

for cell_id, coords in all_add_objs.items():
    all_obj_original[cell_id]['stitched_coords'] = coords

all_remove_objs_list_tmp=all_remove_objs_list.copy()
for obj in all_obj_original:
    if obj in all_remove_objs_list_tmp:
        all_remove_objs_list_tmp.remove(obj)

for obj in all_remove_objs_list_tmp:
    all_obj_original.pop(obj,None)
        

In [ ]:
for obj in all_remove_objs_list:
    all_obj_original.pop(obj,None)

In [ ]:
rem

In [ ]:
pickle.dump(all_obj_original,open(segmentation_output_path / ('segmented_objects_dict.pkl'), 'wb'))

In [ ]:
# Create image
# Calculate image size 

row_min = tile_corners_coords_global_pxl[:,0].min()
col_min = tile_corners_coords_global_pxl[:,1].min()
row_max = tile_corners_coords_global_pxl[:,0].max()
col_max = tile_corners_coords_global_pxl[:,1].max()

nrows = np.int(row_max - row_min + metadata_nuclei['img_height'])
ncols = np.int(col_max - col_min + metadata_nuclei['img_width'])

img = np.zeros([nrows,ncols])

for idx, data_dict in enumerate(all_obj_original.items()):
    coords = data_dict[1]['stitched_coords']
    coords = coords -[row_min,col_min]
    coords = coords.astype(int)
    img[coords[:,0],coords[:,1]] = idx
    
zarr_fpath = segmentation_output_path / 'image_segmented_labels.zarr'
store = zarr.DirectoryStore(zarr_fpath,'w')
grp = zarr.group(store=store,overwrite=True)
grp.create_dataset(name='segmented_labels_image', data=img)

In [ ]:
img.max()

In [ ]:
row_min = tile_corners_coords_global_pxl[:,0].min()
col_min = tile_corners_coords_global_pxl[:,1].min()
row_max = tile_corners_coords_global_pxl[:,0].max()
col_max = tile_corners_coords_global_pxl[:,1].max()

nrows = np.int(row_max - row_min + metadata_nuclei['img_height'])
ncols = np.int(col_max - col_min + metadata_nuclei['img_width'])

img = np.zeros([nrows,ncols])
for idx, data_dict in enumerate(all_obj_original.items()):
    coords = data_dict[1]['stitched_coords']
    coords = coords -[row_min,col_min]
    coords = coords.astype(int)
    img[coords[:,0],coords[:,1]] = idx

In [ ]:
all_add_objs['0-5']

In [ ]:
zarr_fpath = segmentation_output_path / 'image_segmented_labels.zarr'
store = zarr.DirectoryStore(zarr_fpath,'w')
grp = zarr.group(store=store,overwrite=True)
grp.create_dataset(name='segmented_labels_image', data=img)

In [ ]:
all_add_objs

In [ ]:
from pysmFISH.segmentation_NN import Segmenation_NN

In [ ]:
segmentation = Segmenation_NN('stardist')

In [ ]:
mask = segmentation.segment(img)

In [ ]:
from stardist.models import StarDist2D 
from csbdeep.utils import normalize

In [ ]:
#Instantiate model    
model = StarDist2D.from_pretrained('2D_versatile_fluo')
        
#Segment
mask, _ = model.predict_instances(normalize(img),verbose=True)

In [ ]:
model = StarDist2D.from_pretrained('2D_versatile_fluo')

In [ ]:
model.predict_instances

In [ ]:
from cellpose import models

In [ ]:
running_pipeline.tiles_org.reference_corner_fov_position

In [ ]:
running_pipeline.metadata

In [ ]:
running_pipeline.c